Hmwk 4. Built on max's code and my first 3 homeworks

Process:
1. I used the below code to collect training data for 10 hours and saved it as cvs files  in this directory under the folder `training_data`. The `training_data_old` directory has a further 10 hours of data but does not include USDAUD as I collected it prior to the in-class clarification about this currency pair.
2. The first section of the code imports and prepares the data to be used in the regression models
3. Instead of looping over all the currency pairs, I run the regression process (adapted from the pycaret tutorial found in their docs) for each currency pair separately. This was in order to select the best model which varied from one pair to the next.
4. After setting up the models, I tuned, finalised and saved them as per the docs instructions. I then load the models back into the application to be used in the main function.
5. In the main function, every 6 minutes I loop over the currency pairs and replace the vol and fd values with a classificaiton based on the training data. In order to do this I compared the real-time values to the traning data and programmatically added a class based on whether there it was a high, medium or low value.
6. I then use this real-time data each six minutes to make a prediction for the next six minutes, compare it to the actual return and keep note of the error.

In [1]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
import numpy as np

In [2]:
#-------Hmwk4----------
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["EUR","USD",],
                  ["GBP","USD",],
                  ["USD","CHF",],
                  ["USD","CAD",],
                  ["USD","HKD",],
                  ["USD","AUD",],
                  ["USD","NZD",],
                  ["USD","SGD",]]


# Do the necessary imports

import pandas as pd
from pycaret.regression import *

#Create dictionaries to house our raw data, prepared data, models, predictions
dfs = {}
dfs2 ={}
prepared = {}
models = {}
predictions = {}

# Create dictionaries that hold volatility and fd data, we will use these in the main function to assign a class to vol and fd
vol_data = {}
fd_data = {}


#Import our training data from the training_data directory
for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        dfs[f'{from_}{to}'] = pd.read_csv(f'training_data/{from_}{to}_ts.csv')
        dfs2[f'{from_}{to}'] = pd.read_csv(f'training_data/{from_}{to}_ts.csv')
        
        
        # Remove the time period coloumn as it may skew the prediction
        del dfs[f'{from_}{to}']['period']
        
        # sort the values by volatility and assign them into high, medium and low classifications
        dfs[f'{from_}{to}'] = dfs[f'{from_}{to}'].sort_values(by = ["volatility"])
        dfs[f'{from_}{to}'].reset_index(drop=True, inplace=True)
        
        # Saved the ordered vol data to classify real time data points
        vol_data[f'{from_}{to}'] = dfs2[f'{from_}{to}'].loc[0:100, 'volatility']
        
        dfs[f'{from_}{to}'].loc[0:32, 'volatility'] = 1
        dfs[f'{from_}{to}'].loc[33:66, 'volatility'] = 2
        dfs[f'{from_}{to}'].loc[67:100, 'volatility'] = 3
        
        # repeat the process for fractal dimension
        dfs[f'{from_}{to}'] = dfs[f'{from_}{to}'].sort_values(by = ["fd"])
        dfs[f'{from_}{to}'].reset_index(drop=True, inplace=True)
        
        # Saved the ordered fd data to classify real time data points
        fd_data[f'{from_}{to}'] = dfs2[f'{from_}{to}'].loc[0:100, 'fd']
        
        dfs[f'{from_}{to}'].loc[0:32, 'fd'] = 1
        dfs[f'{from_}{to}'].loc[33:66, 'fd'] = 2
        dfs[f'{from_}{to}'].loc[67:100, 'fd'] = 3
        
        # multiply the return by 100000, this will make it easier to make predictions as otherwise our return values are all very closed to 0.
        # we will divide the retrun by 100000 later
        
        dfs[f'{from_}{to}'].loc[0:100, 'return'] = dfs[f'{from_}{to}'].loc[0:100, 'return'] *100000
       
    
        # Add the prepared data for each currency to the prepared dictionary
        prepared[f'{from_}{to}'] = dfs[f'{from_}{to}']
        

In [3]:
# Instead of looping over the currency pairs as I do elsewhere in this project, I run each currency pair in a separate model. This was in order to select the best regression model that varied depending on the currency pair.

# EURUSD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
EURUSD_reg = setup(data = prepared['EURUSD'], target = 'return', session_id=1)



,Description,Value
0,session_id,1
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 20)"


In [4]:
#Select the best model
EURUSD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,31.9228,2034.8274,40.7229,-0.1877,2.0575,3.0768,0.0030
dummy,Dummy Regressor,32.0737,2102.7042,41.5177,-0.2077,2.3930,1.2224,0.0020
lightgbm,Light Gradient Boosting Machine,34.1748,2299.3171,43.6335,-0.3797,1.6860,1.4118,0.0060
llar,Lasso Least Angle Regression,31.6874,1953.3677,40.5609,-0.5037,1.9639,3.8381,0.0030
br,Bayesian Ridge,33.0185,2159.7997,42.9789,-0.6197,2.1231,9.6467,0.0030
ada,AdaBoost Regressor,32.6624,1874.1521,41.1157,-0.7984,1.4773,4.5950,0.0090
knn,K Neighbors Regressor,30.3528,1723.1084,39.3490,-0.8887,1.3286,3.8971,0.0030
lasso,Lasso Regression,31.6275,1920.3609,40.8741,-0.9212,1.6692,11.1408,0.0030
par,Passive Aggressive Regressor,34.7755,2153.0230,43.2641,-1.0286,1.4110,10.2286,0.0030
ridge,Ridge Regression,32.3199,1901.2446,41.1191,-1.1197,1.4534,15.0369,0.0030


In [5]:
# Select the best model
EURUSD_en = create_model("en")

# Tune the rf model
EURUSD_tuned_en = tune_model(EURUSD_en)

# Finalise the model
EURUSD_model = finalize_model(EURUSD_tuned_en)

# Save the model
save_model(EURUSD_model, 'EURUSD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,7.7137,101.1996,10.0598,-0.3284,1.4512,14.9485
1,31.7815,1467.5831,38.3090,-0.0407,2.2132,1.0859
2,27.2720,953.3118,30.8757,0.0786,2.6346,0.9747
3,51.2048,3851.7148,62.0622,0.0341,2.9632,1.0098
4,14.1900,357.5974,18.9102,-0.1175,1.4558,1.5287
5,19.2878,722.9023,26.8868,-0.1011,2.0762,0.8755
6,36.1201,3348.9077,57.8697,0.0528,2.2684,1.0992
7,36.2600,1682.4368,41.0175,-0.0805,2.1207,1.0762
8,36.3453,2204.9146,46.9565,-0.8792,2.2649,1.0484


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  ElasticNet(alpha=1.25, copy_X=True, fit_intercept=True,
  

In [6]:
# GBPUSD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
GBPUSD_reg = setup(data = prepared['GBPUSD'], target = 'return', session_id=2)

,Description,Value
0,session_id,2
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 18)"


In [7]:
#Select the best model
GBPUSD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,30.9705,1592.4619,36.7502,0.0192,1.3947,2.9583,0.0030
br,Bayesian Ridge,32.8381,1937.3917,40.3475,-0.0807,1.4140,1.6636,0.0030
ada,AdaBoost Regressor,33.6091,1989.5272,40.0999,-0.0908,1.3784,2.3639,0.0080
en,Elastic Net,32.7607,1958.3316,41.0677,-0.1124,1.5612,1.7771,0.0030
lightgbm,Light Gradient Boosting Machine,33.0031,2039.6835,41.5174,-0.1207,1.4648,1.8523,0.0030
lr,Linear Regression,32.8256,1790.5813,38.2848,-0.1350,1.5021,3.2921,0.0030
ridge,Ridge Regression,33.9009,1910.6776,40.4411,-0.2153,1.2151,2.1032,0.0030
dummy,Dummy Regressor,34.0540,2090.5188,42.7264,-0.2402,1.3876,2.0348,0.0020
lasso,Lasso Regression,35.2357,2071.5099,42.2928,-0.2478,1.2799,2.1134,0.0030
llar,Lasso Least Angle Regression,35.0134,2158.7241,43.2961,-0.2498,1.5986,1.8436,0.0030


In [8]:
# Select the best model
GBPUSD_knn = create_model("knn")

# Tune the rf model
GBPUSD_tuned_knn = tune_model(GBPUSD_knn)

# Finalise the model
GBPUSD_model = finalize_model(GBPUSD_tuned_knn)

# Save the model
save_model(GBPUSD_model, 'GBPUSD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,47.5616,3535.3123,59.4585,0.1621,1.9651,1.0762
1,25.9603,1489.8531,38.5986,0.2302,1.2339,0.8262
2,39.7515,2068.9995,45.4863,-0.5281,0.8501,1.2813
3,21.6555,560.8217,23.6817,-0.1736,1.2686,2.4671
4,10.9711,159.7343,12.6386,-0.2048,1.5270,4.6318
5,33.3055,1461.2484,38.2263,-0.3703,1.3032,1.1768
6,18.0208,498.3535,22.3238,0.0498,1.4157,0.6723
7,34.9586,2571.4954,50.7099,0.3813,1.4434,0.5194
8,35.4787,1612.0143,40.1499,0.3047,1.5911,2.1939


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('fix_perfect', Remove_100(target='return')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
             

In [9]:
# USDCHF
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDCHF_reg = setup(data = prepared['USDCHF'], target = 'return', session_id=3)

,Description,Value
0,session_id,3
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 21)"


In [10]:
#Select the best model
USDCHF_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,24.9336,1144.4531,32.8069,-0.0239,1.7858,1.5199,0.0030
dummy,Dummy Regressor,25.4670,1178.3193,33.2621,-0.0467,1.6391,1.6327,0.0020
br,Bayesian Ridge,25.4682,1178.3778,33.2631,-0.0468,1.6391,1.6326,0.0030
ada,AdaBoost Regressor,24.4272,1179.8196,32.7237,-0.0499,1.5493,3.1509,0.0080
en,Elastic Net,25.6545,1193.6295,33.4726,-0.0578,1.7613,1.3228,0.0030
omp,Orthogonal Matching Pursuit,25.7880,1209.6768,33.4251,-0.1200,1.3024,3.1987,0.0030
lasso,Lasso Regression,26.7503,1279.9762,34.6305,-0.1326,1.6236,2.2305,0.0030
rf,Random Forest Regressor,25.8293,1265.9184,34.0100,-0.1402,1.4410,4.2432,0.0310
knn,K Neighbors Regressor,26.5857,1240.2414,34.4258,-0.1737,1.3583,3.2660,0.0030
lightgbm,Light Gradient Boosting Machine,27.5179,1312.3554,34.8217,-0.1792,1.3475,2.6512,0.0030


In [11]:
# Select the best model
USDCHF_llar = create_model("llar")

# Tune the rf model
USDCHF_tuned_llar = tune_model(USDCHF_llar)

# Finalise the model
USDCHF_model = finalize_model(USDCHF_tuned_llar)

# Save the model
save_model(USDCHF_model, 'USDCHF_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,26.5079,1104.0019,33.2265,0.0530,1.6260,1.3493
1,19.8865,591.5415,24.3216,-0.1411,1.9986,1.2416
2,17.8091,493.9900,22.2259,0.0473,1.5328,1.1551
3,23.5297,890.3539,29.8388,0.1212,2.0104,0.8750
4,18.0467,722.4450,26.8783,0.0123,1.2986,2.3659
5,22.1979,1176.7726,34.3041,-0.1027,1.8703,3.3748
6,31.8778,2068.9533,45.4857,-0.1265,2.1666,0.8751
7,20.1156,792.3625,28.1489,-0.1982,1.3464,1.0531
8,40.9878,2411.9245,49.1113,0.0101,2.1611,1.9127


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LassoLars(alpha=1.0, copy_X=True, eps=1e-05,
             

In [12]:
# USDCAD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDCAD_reg = setup(data = prepared['USDCAD'], target = 'return', session_id=4)

,Description,Value
0,session_id,4
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 20)"


In [13]:
#Select the best model
USDCAD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,21.1013,774.1978,26.6929,-0.0884,1.4211,1.3357,0.0030
dummy,Dummy Regressor,21.5059,797.0982,27.6424,-0.1677,2.3560,1.0151,0.0020
br,Bayesian Ridge,21.5066,797.1263,27.6429,-0.1678,2.3558,1.0153,0.0030
llar,Lasso Least Angle Regression,21.5956,803.3669,27.7716,-0.1791,2.2916,1.0393,0.0030
en,Elastic Net,21.9168,816.5305,27.9949,-0.2001,2.1468,1.0890,0.0030
rf,Random Forest Regressor,20.6811,813.8613,27.7669,-0.2207,1.1948,1.4677,0.0300
ada,AdaBoost Regressor,21.8611,855.8667,28.1989,-0.2260,1.2732,1.2279,0.0090
lasso,Lasso Regression,22.6006,851.8292,28.5990,-0.2582,2.1216,1.2650,0.0030
lightgbm,Light Gradient Boosting Machine,21.7627,812.9297,28.1404,-0.2595,1.4321,1.1443,0.0030
omp,Orthogonal Matching Pursuit,23.3934,923.4060,29.2484,-0.4030,1.2220,1.5865,0.0020


In [14]:
# Select the best model
USDCAD_knn = create_model("knn")

# Tune the rf model
USDCAD_tuned_knn = tune_model(USDCAD_knn)

# Finalise the model
USDCAD_model = finalize_model(USDCAD_tuned_knn)

# Save the model
save_model(USDCAD_model, 'USDCAD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,22.5702,933.1801,30.5480,-0.6991,2.5358,1.0106
1,21.4619,597.1538,24.4367,0.0063,2.2571,1.0644
2,26.4214,1077.7860,32.8297,-0.0238,2.3339,1.0236
3,29.0860,1608.3534,40.1043,-0.0055,2.1157,1.1992
4,19.1195,536.5604,23.1638,-0.1126,1.6893,0.9507
5,22.1377,569.4546,23.8632,0.0696,2.0199,0.9587
6,15.5569,441.6404,21.0152,-0.0043,2.0008,0.8687
7,16.4394,510.2134,22.5879,-0.8462,1.2900,1.2125
8,20.7568,964.6688,31.0591,-0.2879,1.5780,1.8900


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('fix_perfect', Remove_100(target='return')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
             

In [15]:
# USDHKD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDHKD_reg = setup(data = prepared['USDHKD'], target = 'return', session_id=5)

,Description,Value
0,session_id,5
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 21)"


In [16]:
#Select the best model
USDHKD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,5.7625,73.2732,7.6802,-0.4027,1.1981,2.0179,0.0030
dummy,Dummy Regressor,5.7625,73.2732,7.6802,-0.4027,1.1981,2.0179,0.0020
lasso,Lasso Regression,5.7641,73.3492,7.6839,-0.4037,1.1968,2.0173,0.0030
br,Bayesian Ridge,5.8361,75.4261,7.7802,-0.4311,1.1867,2.0340,0.0030
en,Elastic Net,5.8232,75.0506,7.7712,-0.4401,1.1684,1.8500,0.0030
omp,Orthogonal Matching Pursuit,5.8802,83.8940,8.0880,-0.5713,1.2067,1.2927,0.0030
knn,K Neighbors Regressor,6.0967,81.9599,8.0680,-0.6341,1.0838,1.9014,0.0030
ridge,Ridge Regression,6.1144,85.6704,8.2473,-0.7070,0.9947,2.4571,0.0030
huber,Huber Regressor,6.0701,79.7120,8.0573,-0.7082,0.9591,2.3351,0.0040
rf,Random Forest Regressor,5.7648,83.1191,8.2064,-0.8373,1.0545,1.9240,0.0300


In [17]:
# Select the best model
USDHKD_llar = create_model("llar")

# Tune the rf model
USDHKD_tuned_llar = tune_model(USDHKD_llar)

# Finalise the model
USDHKD_model = finalize_model(USDHKD_tuned_llar)

# Save the model
save_model(USDHKD_model, 'USDHKD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.6893,26.5150,5.1493,-0.3361,0.9328,1.2306
1,6.0566,59.2246,7.6958,-0.1272,1.4848,0.9935
2,5.7719,54.8405,7.4054,-0.1865,1.4734,0.9903
3,3.7984,33.2418,5.7656,-0.1297,0.9896,1.8157
4,4.8502,31.9529,5.6527,-1.1827,0.8260,1.3972
5,5.8222,107.4685,10.3667,-0.4146,0.9825,1.3870
6,13.0000,303.7814,17.4293,-0.4053,2.4271,1.0000
7,2.8571,15.2244,3.9018,-1.1019,0.6703,7.4315
8,7.1419,84.5055,9.1927,-0.0552,1.5234,1.1882


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LassoLars(alpha=1.0, copy_X=True, eps=0.02,
              

In [18]:
# USDAUD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDAUD_reg = setup(data = prepared['USDAUD'], target = 'return', session_id=6)

,Description,Value
0,session_id,6
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 18)"


In [19]:
#Select the best model
USDAUD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,37.2447,2363.2265,47.0874,-0.1569,1.5928,1.1545,0.0030
dummy,Dummy Regressor,37.2445,2363.2074,47.0872,-0.1569,1.5928,1.1545,0.0030
omp,Orthogonal Matching Pursuit,36.6303,2346.9550,46.9831,-0.1727,2.1681,1.3214,0.0020
en,Elastic Net,37.4325,2394.0383,47.4334,-0.1735,1.6660,1.1557,0.0030
llar,Lasso Least Angle Regression,37.2067,2404.0182,47.5651,-0.1865,1.5693,1.0643,0.0030
lasso,Lasso Regression,38.1214,2513.0244,48.6585,-0.2552,1.3881,1.2713,0.0030
par,Passive Aggressive Regressor,41.5029,2669.8634,50.2444,-0.3370,1.4611,1.9809,0.0020
knn,K Neighbors Regressor,37.3701,2504.9735,48.6218,-0.3423,1.0673,1.1604,0.0030
ridge,Ridge Regression,39.2173,2690.1724,50.1141,-0.3436,1.3302,1.4237,0.0030
rf,Random Forest Regressor,34.1219,1994.3982,43.5074,-0.3452,1.1388,1.8101,0.0300


In [20]:
# Select the best model
USDAUD_br = create_model("br")

# Tune the rf model
USDAUD_tuned_br = tune_model(USDAUD_br)

# Finalise the model
USDAUD_model = finalize_model(USDAUD_tuned_br)

# Save the model
save_model(USDAUD_model, 'USDAUD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,39.9129,2211.5726,47.0274,-0.0098,1.6231,1.2295
1,30.8426,1883.1641,43.3954,-0.1231,1.3250,1.6644
2,38.5257,2872.2162,53.5931,-0.1740,1.4067,0.8490
3,35.5997,1629.9127,40.3722,-0.0937,1.7485,1.0300
4,19.0793,454.9958,21.3306,-0.2556,1.2504,0.9387
5,35.8679,1854.0454,43.0586,-0.0865,1.7885,0.8987
6,48.1528,2690.5922,51.8709,-0.2140,1.5367,1.1216
7,52.4740,5026.1885,70.8956,-0.0271,1.7340,0.9399
8,36.5407,1890.9546,43.4851,-0.0002,1.5527,1.5957


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  BayesianRidge(alpha_1=0.3, alpha_2=0.15, alpha_init=None,


In [21]:
# USDNZD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDNZD_reg = setup(data = prepared['USDNZD'], target = 'return', session_id=7)

,Description,Value
0,session_id,7
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 19)"


In [22]:
#Select the best model
USDNZD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
llar,Lasso Least Angle Regression,33.0066,1972.2332,43.0093,-0.0920,2.0775,7.4199,0.0030
en,Elastic Net,33.1218,2056.2462,43.7568,-0.1066,1.6301,10.4271,0.0020
dummy,Dummy Regressor,32.9008,2088.1273,44.1798,-0.1319,1.6075,12.6019,0.0020
br,Bayesian Ridge,33.1781,2112.0539,44.4157,-0.1426,1.6037,12.1540,0.0030
omp,Orthogonal Matching Pursuit,32.8090,1945.1269,42.9724,-0.1479,2.3076,2.4022,0.0020
lasso,Lasso Regression,34.1592,1998.3514,43.4080,-0.1487,1.8558,9.9541,0.0030
ridge,Ridge Regression,34.8395,2089.4185,44.6055,-0.2240,1.5712,10.2369,0.0020
par,Passive Aggressive Regressor,34.6700,2204.5086,45.3538,-0.2428,1.2632,34.3830,0.0030
ada,AdaBoost Regressor,34.4615,1986.6512,43.8325,-0.2434,1.6104,16.6019,0.0100
lightgbm,Light Gradient Boosting Machine,35.2194,2320.2502,46.5011,-0.2631,1.7241,7.9758,0.0030


In [23]:
# Select the best model
USDNZD_llar = create_model("llar")

# Tune the rf model
USDNZD_tuned_llar = tune_model(USDNZD_llar)

# Finalise the model
USDNZD_model = finalize_model(USDNZD_tuned_llar)

# Save the model
save_model(USDNZD_model, 'USDNZD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,28.5351,1167.4529,34.1680,-0.1081,2.4304,6.2428
1,24.7928,879.1345,29.6502,-0.0774,1.8637,1.0314
2,31.7988,2414.3171,49.1357,-0.0760,1.8765,3.0338
3,45.9515,4704.6842,68.5907,0.0926,1.8012,3.5522
4,38.0362,2486.6901,49.8667,-0.0870,2.5435,1.0408
5,37.8324,2339.6058,48.3695,-0.3086,3.0139,0.9476
6,27.1823,947.3382,30.7789,0.2770,1.7757,0.9726
7,28.5223,1477.1472,38.4337,-0.1849,1.7667,1.1211
8,34.6182,1891.9728,43.4968,0.0952,1.9655,54.3601


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LassoLars(alpha=1.0, copy_X=True, eps=0.0001,
            

In [24]:
# USDSGD
# Below code creates the regresion model, most of it is adapted from the pycaret tutorial

        
# Set up the regression
USDSGD_reg = setup(data = prepared['USDSGD'], target = 'return', session_id=8)

,Description,Value
0,session_id,8
1,Target,return
2,Original Data,"(100, 7)"
3,Missing Values,False
4,Numeric Features,5
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(69, 18)"


In [25]:
#Select the best model
USDSGD_best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,21.5307,753.4775,26.5524,-0.1581,1.4319,1.9124,0.0020
dummy,Dummy Regressor,21.5303,753.4648,26.5522,-0.1581,1.4320,1.9133,0.0020
llar,Lasso Least Angle Regression,21.5652,755.3874,26.5808,-0.1598,1.4297,1.9165,0.0020
lightgbm,Light Gradient Boosting Machine,21.2356,730.0384,26.0533,-0.1609,1.4767,1.6487,0.0030
knn,K Neighbors Regressor,21.9921,763.4031,26.5532,-0.1757,1.3091,1.8926,0.0030
en,Elastic Net,21.8839,769.1063,26.7959,-0.1795,1.4523,1.5001,0.0030
lasso,Lasso Regression,22.8068,830.3397,27.7026,-0.2571,1.5543,1.4838,0.0020
omp,Orthogonal Matching Pursuit,22.2835,825.0605,27.6289,-0.2638,1.4303,1.4448,0.0020
lr,Linear Regression,22.4061,841.8236,28.2810,-0.4780,1.3174,2.4881,0.0020
ridge,Ridge Regression,24.8864,985.5284,30.0012,-0.4986,1.2486,1.7819,0.0020


In [26]:
# Select the best model
USDSGD_br = create_model("br")

# Tune the rf model
USDSGD_tuned_br = tune_model(USDSGD_br)

# Finalise the model
USDSGD_model = finalize_model(USDSGD_tuned_br)

# Save the model
save_model(USDSGD_model, 'USDSGD_model')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,22.9156,819.4753,28.6265,-0.8262,2.3179,0.9505
1,25.9592,1133.4738,33.6671,-0.0123,1.8943,1.0554
2,24.1526,757.1745,27.5168,-0.0379,1.9392,1.0049
3,17.4349,612.1017,24.7407,-0.0240,1.5162,1.1367
4,13.7578,311.1584,17.6397,-0.0010,1.4085,1.3096
5,10.2698,157.5569,12.5522,-0.1285,1.1191,1.3586
6,19.7594,594.3433,24.3792,-0.3078,1.3666,4.1825
7,34.3638,1502.0254,38.7560,-0.0059,2.0377,0.9672
8,24.7495,998.4245,31.5979,-0.1826,2.1689,0.8670


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='return',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy=...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  BayesianRidge(alpha_1=0.001, alpha_2=0.001, alpha_init=Non

In [27]:
# Load the models ready to be used in the main function
loaded_models = {}
loaded_models["EURUSD"] = load_model('EURUSD_model')
loaded_models["GBPUSD"] = load_model('GBPUSD_model')
loaded_models["USDCHF"] = load_model('USDCHF_model')
loaded_models["USDCAD"] = load_model('USDCAD_model')
loaded_models["USDHKD"] = load_model('USDHKD_model')
loaded_models["USDAUD"] = load_model('USDAUD_model')
loaded_models["USDNZD"] = load_model('USDNZD_model')
loaded_models["USDSGD"] = load_model('USDSGD_model')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [32]:
# The below are helper functions for the main function
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg(inserttime text, avgfxrate  numeric, stdfxrate numeric);"))

# Create a table for storing data required for hmwk2 + 3
def intialize_ts_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_ts(inserttime text, period text, maximum numeric, minimum numeric, mean numeric, volatility numeric, fd numeric, return numeric);"))

# Create a table for storing the results from the stop loss strategy
def intialize_results_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_results(inserttime text, period text, position text, balance numeric, profitloss numeric, status text);"))
            
# Create a table for storing the predictions from the regression model
def intialize_predictions_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_predictions(inserttime text, predictedreturn numeric, actualreturn text numeric, error numeric);"))
            
            
# This function is called every 6 minutes to aggregate the data, store it in the aggregate table, 
# and then delete the raw data
def aggregate_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            result = conn.execute(text("SELECT AVG(fxrate) as avg_price, COUNT(fxrate) as tot_count FROM "+curr[0]+curr[1]+"_raw;"))
            for row in result:
                avg_price = row.avg_price
                tot_count = row.tot_count
            std_res = conn.execute(text("SELECT SUM((fxrate - "+str(avg_price)+")*(fxrate - "+str(avg_price)+"))/("+str(tot_count)+"-1) as std_price FROM "+curr[0]+curr[1]+"_raw;"))
            for row in std_res:
                std_price = sqrt(row.std_price)
            date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM "+curr[0]+curr[1]+"_raw;"))
            for row in date_res:
                last_date = row.last_date
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_agg (inserttime, avgfxrate, stdfxrate) VALUES (:inserttime, :avgfxrate, :stdfxrate);"),[{"inserttime": last_date, "avgfxrate": avg_price, "stdfxrate": std_price}])
            
            # This calculates and stores the return values
            exec("curr[2].append("+curr[0]+curr[1]+"_return(last_date,avg_price))")
            #exec("print(\"The return for "+curr[0]+curr[1]+" is:"+str(curr[2][-1].hist_return)+" \")")
            
            if len(curr[2]) > 5:
                try:
                    avg_pop_value = curr[2][-6].hist_return
                except:
                    avg_pop_value = 0
                if isnan(avg_pop_value) == True:
                    avg_pop_value = 0
            else:
                avg_pop_value = 0
            # Calculate the average return value and print it/store it
            curr_avg = curr[2][-1].get_avg(avg_pop_value)
            #exec("print(\"The average return for "+curr[0]+curr[1]+" is:"+str(curr_avg)+" \")")
            
            # Now that we have the average return, loop through the last 5 rows in the list to start compiling the 
            # data needed to calculate the standard deviation
            for row in curr[2][-5:]:
                row.add_to_running_squared_sum(curr_avg)
            
            # Calculate the standard dev using the avg
            curr_std = curr[2][-1].get_std()
            #exec("print(\"The standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_std)+" \")")
            
            # Calculate the average standard dev
            if len(curr[2]) > 5:
                try:
                    pop_value = curr[2][-6].std_return
                except:
                    pop_value = 0
            else:
                pop_value = 0
            curr_avg_std = curr[2][-1].get_avg_std(pop_value)
            #exec("print(\"The average standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_avg_std)+" \")")
            

#                    ----------------- Hmwk 3 -----------------------


# This funtion initialises trading by assigning 5 currencies long positions and 5 currencies short positions.
# It adds 100 to their balance.

def initialise_trading(engine, currency_pairs, dic):
    i = 0
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        i +=1
        
        if (i % 2 != 0):
            dic[f'{from_}{to} position'] = 'long'
            dic[f'{from_}{to} balance'] = 100
            dic[f'{from_}{to} trade_status'] = 'live'
            
        
        if (i % 2 == 0):
            dic[f'{from_}{to} position'] = 'short'
            dic[f'{from_}{to} balance'] = 100
            dic[f'{from_}{to} trade_status'] = 'live'
            

# This function contains the stop loss strategy as outlined in the assignment 
            
def stop_loss_strategy(engine, currency_pairs, dic, hours_past):
    
    # This function gets called every hour

    # Loop through each currency pair
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        
        
    
    # We can reference the acceptable loss from this logic array. This refactoring saved many lines of copy and pasted code.
    # Each element is an acceptable loss, it's index + 1 is its corresponding phase. I.e 0.250 is acceptable in the first hour, 0.100 is acceptable in the second hour.
    
        logic = [0.250, 0.150, 0.100, 0.050, 0.050, 0.050, 0.050, 0.050, 0.050, 0.050]
            
               
        # If 10 hours have past, exit the trades and compute the profit/loss
        if hours_past == 10:
            # Exit trades and compute balance, profit or loss
            dic[f'{from_}{to} trade_status'] = 'exited'
            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
            
            if (dic[f'{from_}{to} position'] == 'long'):
                dic[f'{from_}{to} profit_loss'] = (dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
            
            if (dic[f'{from_}{to} position'] == 'short'):
                dic[f'{from_}{to} profit_loss'] = -(dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
                
            
                
        else:
            # Follow the trading logic   
            for index, val in enumerate(logic):
                #If first period
                if hours_past == index+1:
                    #If long
                    if (dic[f'{from_}{to} position'] == 'long') & (dic[f'{from_}{to} trade_status'] == 'live'):
                    
                        # If long and loss is larger than accepted loss compute loss and close trade
                        if ((dic[f'{from_}{to} return']) < -(logic[index]/100)):
                            dic[f'{from_}{to} trade_status'] = 'exited'
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
                            

                        # If long and loss is less than accepted loss, compute profit or loss, add it to the position and add another 100 to the trade 
                        if ((dic[f'{from_}{to} return']) >= -(logic[index]/100)):
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] + (dic[f'{from_}{to} balance'] * dic[f'{from_}{to} return']) + 100
                            dic[f'{from_}{to} total_invested'] += 100
                        
                        dic[f'{from_}{to} profit_loss'] = (dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
                        
                        
                    if (dic[f'{from_}{to} position'] == 'short') & (dic[f'{from_}{to} trade_status'] == 'live'):
                         # If short and gain is larger than accepted gain compute loss and close trade
                        if ((dic[f'{from_}{to} return']) > (logic[index]/100)):
                            dic[f'{from_}{to} trade_status'] = 'exited'
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] * (1 - dic[f'{from_}{to} return'])
                            
                         # If short and gain is less than accepted gain compute profit or loss add it to the position and add another 100 to the trade
                        if ((dic[f'{from_}{to} return']) <= (logic[index]/100)):
                            dic[f'{from_}{to} balance'] = dic[f'{from_}{to} balance'] + (dic[f'{from_}{to} balance'] * dic[f'{from_}{to} return']) + 100
                            dic[f'{from_}{to} total_invested'] += 100

                        dic[f'{from_}{to} profit_loss'] = -(dic[f'{from_}{to} balance'] - dic[f'{from_}{to} total_invested'])
        
        
        
        # Get the current time and format it
        insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Write the results into the db
        with engine.begin() as conn:
            conn.execute(text("INSERT INTO "+from_+to+"_results(inserttime, period, position, balance, profitloss, status) VALUES (:inserttime, :period, :position, :balance, :profitloss, :status)"),[{"inserttime": insert_time, "period": hours_past, "position": dic[f'{from_}{to} position'], "balance": dic[f'{from_}{to} balance'], "profitloss": dic[f'{from_}{to} profit_loss'], "status": dic[f'{from_}{to} trade_status']}])
                    
        
        
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main(currency_pairs):
    # The api key given by the professor
    key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"
    
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    hour_count = 0
    hours_past = 0
    
    # Create a dictionary of variables that will act as local storage for the various currency paris
    dic = {}
    
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
        
        #Initialise the variables that we will need
        dic[f'{from_}{to} maximum'] = float('-inf')
        dic[f'{from_}{to} minimum'] = float('inf')
        dic[f'{from_}{to} prices'] = []
        dic[f'{from_}{to} running_total'] = 0
        dic[f'{from_}{to} cross_count'] = 0
        dic[f'{from_}{to} period'] = 1
        dic[f'{from_}{to} keltner_upper_bands'] = []
        dic[f'{from_}{to} keltner_lower_bands'] = []
        dic[f'{from_}{to} avg_price'] = None
        dic[f'{from_}{to} last_price'] = None
        dic[f'{from_}{to} upper_count'] = 0
        dic[f'{from_}{to} lower_count'] = 0
        dic[f'{from_}{to} fd'] = 0
        dic[f'{from_}{to} old_mean'] = 0
        dic[f'{from_}{to} mean'] = 0
        dic[f'{from_}{to} return'] = 0
        # variables that will keep track of investments
        dic[f'{from_}{to} balance'] = 0       # we will initialise this to 100
        dic[f'{from_}{to} profit_loss'] = 0
        dic[f'{from_}{to} total_invested'] = 100
        dic[f'{from_}{to} number_of_hours'] = 0
        dic[f'{from_}{to} trade_status'] = '' # live or exited
        dic[f'{from_}{to} position'] = ''     # long or short
        dic[f'{from_}{to} predicted'] = 0     # predicted return from the model
        dic[f'{from_}{to} error'] = 0         # error from prediction
        
        
    
    
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite+pysqlite:///sqlite/final.db", echo=False, future=True)
    
    
   
        
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    intialize_ts_tables(engine,currency_pairs)
    intialize_results_tables(engine,currency_pairs)
    intialize_predictions_tables(engine,currency_pairs)
    
    
    
    
    # Start trading!
    initialise_trading(engine, currency_pairs, dic)
    
    # Open a RESTClient for making the api calls
    with RESTClient(key) as client:
        # Loop that runs until the total duration of the program hits 24 hours. 
        while count < 86400: # 86400 seconds = 24 hours
            
            
            # Make a check to see if 6 minutes has been reached or not
            if agg_count == 360:
                
                
                # Aggregate the data and clear the raw data tables
                #aggregate_raw_data_tables(engine,currency_pairs)
                reset_raw_data_tables(engine,currency_pairs)
                agg_count = 0
            
            
            # Every ten hours run the trailing stop strategy (3600 seconds)
            if hour_count == 3600:

                
                # Run the stop loss strategy
                stop_loss_strategy(engine, currency_pairs, dic, hours_past)
                
                 # Increment hours that have past
                hours_past += 1

                # Reset hour count
                hour_count = 0
                
               

            # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
            # code takes about .15 seconds to run
            time.sleep(0.5)
            
            # Increment the counters
            count += 1
            agg_count += 1
            hour_count += 1

            # Loop through each currency pair
            for currency in currency_pairs:
                # Set the input variables to the API
                from_ = currency[0]
                to = currency[1]
                
                if agg_count == 360:
            
                    # Every six minutes...
                    
                    # Calculate the mean
                    prices_arr = np.array(dic[f'{from_}{to} prices'])
                    dic[f'{from_}{to} running_total'] = prices_arr.sum()
                    dic[f'{from_}{to} mean'] = dic[f'{from_}{to} running_total'] / len(prices_arr)
                    
                    # Calculate the return
                    if  dic[f'{from_}{to} period'] > 1:
                        dic[f'{from_}{to} return'] = (dic[f'{from_}{to} mean'] - dic[f'{from_}{to} old_mean']) / dic[f'{from_}{to} old_mean']
                    else:
                        dic[f'{from_}{to} return'] = 0
                        
                        
                    # Assign the previous mean to the old_mean
                    dic[f'{from_}{to} old_mean'] = dic[f'{from_}{to} mean']

                    
                    # Reset values
                    dic[f'{from_}{to} prices'] = []
                    dic[f'{from_}{to} running_total'] = 0
    
        
                    # Write the volatility info to database
                
                    with engine.begin() as conn:
                        conn.execute(text("INSERT INTO "+from_+to+"_ts(inserttime, period, maximum, minimum, mean, volatility, fd, return) VALUES (:inserttime, :period, :maximum, :minimum, :mean, :volatility, :fd, :return)"),[{"inserttime": insert_time, "period": dic[f'{from_}{to} period'], "maximum": dic[f'{from_}{to} maximum'], "minimum": dic[f'{from_}{to} minimum'], "mean": dic[f'{from_}{to} mean'], "volatility": dic[f'{from_}{to} vol'], "fd": dic[f'{from_}{to} fd'], "return": dic[f'{from_}{to} return']}])
                    
                    
                    # Write the prediction info into the db
                    
                    dic[f'{from_}{to} error'] = dic[f'{from_}{to} return'] - dic[f'{from_}{to} predicted']
                    
                    
                    with engine.begin() as conn:
                        conn.execute(text("INSERT INTO "+from_+to+"_predictions(inserttime, predictedreturn, actualreturn, error) VALUES (:inserttime, :predictedreturn, :actualreturn, :error)"),[{"inserttime": insert_time, "predictedreturn": dic[f'{from_}{to} predicted'], "actualreturn": dic[f'{from_}{to} return'], "error": dic[f'{from_}{to} error'] }])
                                       
                    
                    # Run the prediction for next period
                    
                    data = {'inserttime': [insert_time], 'maximum': [dic[f'{from_}{to} maximum']], 'minimum': [dic[f'{from_}{to} minimum']],'mean': [dic[f'{from_}{to} mean']], 'volatility': [dic[f'{from_}{to} vol']], 'fd': [dic[f'{from_}{to} fd']], 'return': [dic[f'{from_}{to} return']] }
                    df = pd.DataFrame(data=data)
                    predictions[f'{from_}{to}'] = predict_model(loaded_models[f'{from_}{to}'], data = df)

                    # Save the prediction to the dictionary                    
                    dic[f'{from_}{to} predicted'] = predictions[f'{from_}{to}'].loc[0]['Label'] / 100000
                    
                    
                    
                    # Reset the currency specific variables
                    dic[f'{from_}{to} maximum'] = float('-inf')
                    dic[f'{from_}{to} minimum'] = float('inf')
                    dic[f'{from_}{to} cross_count'] = 0 
                    dic[f'{from_}{to} period'] +=1

                    dic[f'{from_}{to} keltner_upper_bands'] = []
                    dic[f'{from_}{to} keltner_lower_bands'] = []
                    dic[f'{from_}{to} upper_count'] = 0
                    dic[f'{from_}{to} lower_count'] = 0

                    

                    # If the first period has passed, calculate the keltner bands
                    if dic[f'{from_}{to} period'] > 1:

                    # Create 100 upper Keltner bands
                        for num in range(100):
                            calc = dic[f'{from_}{to} mean'] + num*0.025*dic[f'{from_}{to} vol']
                            dic[f'{from_}{to} keltner_upper_bands'].append(calc)
                            
                    # Create 100 lower Keltner bands
                        for num in range(100):
                            calc = dic[f'{from_}{to} mean'] - num*0.025*dic[f'{from_}{to} vol']
                            dic[f'{from_}{to} keltner_lower_bands'].append(calc)
                        
                

                # Call the API with the required parameters
                try:
                    resp = client.forex_currencies_real_time_currency_conversion(from_, to, amount=100, precision=2)
                except:
                    continue

                # This gets the Last Trade object defined in the API Resource
                last_trade = resp.last

                # Format the timestamp from the result
                dt = ts_to_datetime(last_trade["timestamp"])

                # Get the current time and format it
                insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
                # Assign the old average price to the last price
                dic[f'{from_}{to} last_price'] = dic[f'{from_}{to} avg_price']
                
                
                # Calculate the new average price by taking the average of the bid and ask prices
                avg_price = (last_trade['bid'] + last_trade['ask'])/2
                dic[f'{from_}{to} avg_price'] = (last_trade['bid'] + last_trade['ask'])/2
                
                # Calculate the max price in the past six minutes
                if dic[f'{from_}{to} avg_price'] > dic[f'{from_}{to} maximum']:
                    dic[f'{from_}{to} maximum'] = dic[f'{from_}{to} avg_price']
                            
                # Calculate the min price in the last six minutes
                if dic[f'{from_}{to} avg_price'] < dic[f'{from_}{to} minimum']:
                    dic[f'{from_}{to} minimum'] = dic[f'{from_}{to} avg_price']
                
                # Calculate the volatility over the last six minutes
                dic[f'{from_}{to} vol'] = dic[f'{from_}{to} maximum'] - dic[f'{from_}{to} minimum']
                
                # Replace the vol with a class (1,2,3)
                vol_arr = vol_data[f'{from_}{to}'].to_numpy().tolist()
                vol_arr.append(dic[f'{from_}{to} vol'])
                vol_r = vol_arr.index(dic[f'{from_}{to} vol'])
                if vol_r <= 32:
                    dic[f'{from_}{to} vol'] = 1
                    
                if (vol_r > 32) &  vol_r <= 66:
                    dic[f'{from_}{to} vol'] = 2
                
                if (vol_r > 66):
                    dic[f'{from_}{to} vol'] = 3
                    
                
                # Calculate the fractal dimension
                # For each new price we want to know how many bands have been crossed. 
        
                upper = np.array(dic[f'{from_}{to} keltner_upper_bands'])
                lower = np.array(dic[f'{from_}{to} keltner_lower_bands'])
                
                # How many numbers in the keltner band are greater than the old price and less than the new price
                if dic[f'{from_}{to} last_price'] is not None:
                    dic[f'{from_}{to} upper_count'] = ((dic[f'{from_}{to} last_price'] < upper) & (upper < dic[f'{from_}{to} avg_price'])).sum()

                # How many numbers in the keltner band are less than the old price and greater than the new price
                if dic[f'{from_}{to} last_price'] is not None:
                    dic[f'{from_}{to} lower_count'] = ((dic[f'{from_}{to} last_price'] > lower) & (lower > dic[f'{from_}{to} avg_price'])).sum()

                # Add the above counts from upper and lower bands together
                dic[f'{from_}{to} cross'] = dic[f'{from_}{to} upper_count'] + dic[f'{from_}{to} lower_count']
                
                # Add the total to the running total of crosses over the six minute period
                dic[f'{from_}{to} cross_count'] = dic[f'{from_}{to} cross_count'] +  dic[f'{from_}{to} cross']
                
                # Divide the cross_count by the volatility in order to calculte the fractal dimenstion
                if  dic[f'{from_}{to} vol'] != 0:
                     dic[f'{from_}{to} fd'] = dic[f'{from_}{to} cross_count'] /  dic[f'{from_}{to} vol']
                
                # Replace the fd with a class (1,2,3)
                fd_arr = fd_data[f'{from_}{to}'].to_numpy().tolist()
                fd_arr.append(dic[f'{from_}{to} fd'])
                fd_r = fd_arr.index(dic[f'{from_}{to} fd'])
                if fd_r <= 32:
                    dic[f'{from_}{to} fd'] = 1
                    
                if (fd_r > 32) &  fd_r <= 66:
                    dic[f'{from_}{to} fd'] = 2
                
                if (fd_r > 66):
                    dic[f'{from_}{to} fd'] = 3
                
                # Keep track of prices over last 6 minutes
                (dic[f'{from_}{to} prices']).append(dic[f'{from_}{to} avg_price'])
                
                
                # Write the data to the SQLite database, raw data tables
                with engine.begin() as conn:
                    conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])
                
                
                    

In [33]:
# Run the main data collection loop
main(currency_pairs)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Elastic Net,1.5704,2.4662,1.5704,nan,0.9441,nan


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,2.8572,8.1636,2.8572,nan,1.3499,nan


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,2.0259,4.1044,2.0259,nan,1.1072,nan


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,4.8847,23.8599,4.8847,nan,1.7723,nan


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,0.3078,0.0948,0.3078,nan,0.2684,nan


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,0.3081,0.0950,0.3081,nan,0.2686,25783.9536


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,7.5131,56.4468,7.5131,nan,2.1412,22369.7525


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Least Angle Regression,5.4797,30.0276,5.4797,nan,1.8684,26536.6544


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,1.8481,3.4153,1.8481,nan,1.0466,196082.9725


KeyboardInterrupt: 